In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from pandas import read_csv
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.metrics import mean_absolute_percentage_error


data_path = Path("data") / "electricityPrediction.csv"
df = read_csv(data_path, header=None, names=['V1','V2','V3','V4','V5','V6','V7'])
# 1. Feature Engineering (keeping the sin/cos features from Linear Regression)
X = df.iloc[:, 0:6].copy()
t = np.arange(len(df))
X['sin_t'] = np.sin(2 * np.pi * t / 24)
X['cos_t'] = np.cos(2 * np.pi * t / 24)
X.columns = X.columns.astype(str)

# 2. Log Transform Target
y = np.log(df.iloc[:, 6])

# 3. Scaling (Essential for Neural Nets)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# 4. Deep Architecture (5, 5, 5) from PPT
mlp_deep = MLPRegressor(
    hidden_layer_sizes=(5, 5, 5), 
    activation='logistic', 
    solver='sgd', 
    learning_rate_init=0.01,
    max_iter=2000, 
    random_state=42
)

mlp_deep.fit(X_train, y_train)

# 5. Inverse Transform for MAPE
predictions = np.exp(mlp_deep.predict(X_test))
actuals = np.exp(y_test)
print(f"Deep MLP (5,5,5) MAPE: {mean_absolute_percentage_error(actuals, predictions):.2f}%")

Deep MLP (5,5,5) MAPE: 0.12%
